# ASync Example Bibliograpy Explorer

In [ ]:
import grpc
import json
from example_bib_app.grpc import example_bib_app_pb2_grpc, example_bib_app_pb2

from datetime import datetime

In [ ]:
with grpc.insecure_channel("localhost:50051") as channel:
        stub = example_bib_app_pb2_grpc.AuthorControllerStub(channel)
        
        response = stub.List(example_bib_app_pb2.AuthorListRequest())

        print(response)
        

In [ ]:
# gRPC clients

grpc_server_host = 'localhost' # '127.0.0.1' 
grpc_server_port = 50051

sync_channel = grpc.insecure_channel(f'{grpc_server_host}:{grpc_server_port}')
author_client = example_bib_app_pb2_grpc.AuthorControllerStub(sync_channel)
publisher_client = example_bib_app_pb2_grpc.PublisherControllerStub(sync_channel)

book_client = example_bib_app_pb2_grpc.BookControllerStub(sync_channel)

In [ ]:
author_client.List(example_bib_app_pb2.AuthorListRequest())

## Creating Authors

In [ ]:
author_response = author_client.Create(example_bib_app_pb2.AuthorRequest(name_first='John', name_last='Doe3', birth_date=datetime.now().strftime('%Y-%m-%d')))
author_response

In [ ]:
author_response.author_id

In [ ]:
publisher_response = publisher_client.Create(example_bib_app_pb2.PublisherRequest(name='Hopper', website='https://hopper.com' ))
publisher_response

In [ ]:
# creating books

for i in range(10):
    book_client.Create(example_bib_app_pb2.BookRequest(title=f'book {i}', 
                                                            authors=[author_response.author_id],
                                                            isbn=f'isbn-{i}',
                                                            publisher=publisher_response.publisher_id,
                                                            publication_date=datetime.now().strftime('%Y-%m-%d')
                                                            ))

## Using Filters

In [ ]:
filter_as_dict = {"name_last": "Doe"}
metadata = (("filters", (json.dumps(filter_as_dict))),)

res = author_client.List(example_bib_app_pb2.AuthorListRequest(), metadata=metadata)

res

In [ ]:
str(filter_as_dict)

In [ ]:
json.dumps(filter_as_dict)

## Closing channels

In [ ]:
sync_channel.close()